In [1666]:
import numpy as np
import time

In [1667]:
x=np.random.rand(12).reshape(4,3)
x

array([[  1.39500477e-01,   5.61470802e-01,   9.72927769e-01],
       [  4.80787931e-01,   3.43819375e-02,   2.82004834e-01],
       [  9.20560882e-01,   9.71241034e-01,   6.05636754e-01],
       [  5.89425027e-01,   8.55702125e-01,   5.07748149e-04]])

In [1668]:
y=np.random.randint(0,2,4).reshape(4,1)

In [1669]:
theta=(np.random.randn(15)*0.1).reshape(5,3)
b=np.random.rand(5).reshape(5,1)
print (theta)
print (b)

[[ 0.07974715  0.07760073  0.23053318]
 [-0.06290633 -0.02095342  0.08009875]
 [ 0.00092499 -0.08012218 -0.08876821]
 [-0.15600683 -0.12057702 -0.14562559]
 [ 0.0829836   0.10180823 -0.06107614]]
[[ 0.54246289]
 [ 0.52506944]
 [ 0.30295833]
 [ 0.15221704]
 [ 0.96576197]]


# Forward Propogation

In [1670]:
#dimensions of each layer
dimensions=np.array([5,4,3,2,1])

#empty array of weights of all layers 
weights=np.zeros(len(dimensions)*(dimensions[0]**2)).reshape(len(dimensions),(dimensions[0]**2))

#empty array of biases of all layers
bias=np.zeros(len(dimensions)*dimensions[0]).reshape(len(dimensions),dimensions[0])

#empty array of Zs of all layers
Z=np.zeros(len(dimensions)*(dimensions[0]**2)).reshape(len(dimensions),(dimensions[0]**2))

#empty array of as of all layers
A=np.zeros(len(dimensions)*(dimensions[0]**2)).reshape(len(dimensions),dimensions[0]**2)

#empty array for shapes of a
shapes=np.zeros(10).reshape(5,2)

#empty array for shapes of thetas
shapes_thetas=np.zeros(10).reshape(5,2)

#empty array for shapes of bias
shapes_bias=np.zeros(10).reshape(5,2)

tic=time.time()
#loop for computation of z, a, theta and bias  of all layers expect input
for i in range (0,len(dimensions)):
    # array of z of each layer
    z=np.zeros(dimensions[i]*len(x)).reshape(dimensions[i],len(x))
    
    
    #print (theta.shape)
    shapes_thetas[i]=theta.shape
    
    shapes_bias[i]=b.shape
    #store all layer thetas in one array
    k=0
    for m in range(0,len(theta)):
        for n in range(0,len(theta[0])):
            weights[i][k]=theta[m][n]
            k+=1
    
    #store all layer biases in one array        
    c=0
    for f in range(0,len(b)):
        bias[i][c]=b[f][0]
        c+=1
    
    #compute z of each layer
    for j in range(0,len(theta)):
        z[j]=np.dot(x,theta[j].T)+b[j]
    z=z.T
    #print (z.shape)
    #store all layers Zs in one array
    temp=0
    for m in range(0,len(z)):
        for n in range(0,len(z[0])):
            Z[i][temp]=z[m][n]
            temp+=1
    
    #compute a 
    a=1/(1+np.exp(-z))
    shapes[i]=a.shape
    if i==(len(dimensions)-1):
        a=-y*np.log(a)+(1-y)*np.log(1-a)
    #store all layers As in one array 
    temp=0
    for m in range(0,len(a)):
        for n in range(0,len(a[0])):
            A[i][temp]=a[m][n]
            temp+=1
    
    #make output of each layer as input of another layer
    x=a
    
    #Go outside the loop 
    if i==(len(dimensions)-1):
        break
        
    #generate new thetas and biases for each layer
    theta=(np.random.randn(dimensions[i+1]*dimensions[i])*0.1).reshape(dimensions[i+1],dimensions[i])
    b=np.random.rand(dimensions[i+1]).reshape(dimensions[i+1],1) 
toc=time.time()
print ((toc-tic)*1000,'nanoseconds')

0.0 nanoseconds


# Backword Propogation

In [1671]:
da=-y/a+(1-y)/(1-a) 
for i in range (len(dimensions)-1,-1,-1):
    a=np.trim_zeros(A[i])
    a=a.reshape(int(shapes[i][0]),int(shapes[i][1]))
    b=np.trim_zeros(A[i-1])
    b=b.reshape(int(shapes[i-1][0]),int(shapes[i-1][1]))
    z1=np.trim_zeros(Z[i])
    z1=z1.reshape(int(shapes[i][0]),int(shapes[i][1]))
    dz=da*(a*(1-a))
    dw=np.dot(b.T,dz)
    dw=np.mean(dw,axis=0)
    db=np.mean(dz,axis=0)
    w=np.trim_zeros(weights[i])
    w=w.reshape(int(shapes_thetas[i][0]),int(shapes_thetas[i][1]))
    bi=np.trim_zeros(bias[i])
    bi=bi.reshape(int(shapes_bias[i][0]),int(shapes_bias[i][1]))
    for j in range(0,len(w)):
        w[j]=w[j]-0.021*dw[j]
    for l in range(0,len(bi)):
        bi[l]=bi[l]-0.021*db[l]   
    da=np.dot(dz,w)